<a href="https://colab.research.google.com/github/EverestRabaudi/Vedant-Coding-Project/blob/main/Vedant_Project_Website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Basketball Stats Predictor</title>
    <link rel="stylesheet" href="styles.css">
</head>
<body>
    <div class="container">
        <h1>Basketball Player Stats Input</h1>
        <form id="statsForm">
            <label for="playerName">Player Name:</label>
            <input type="text" id="playerName" required>

            <label for="pts">Points (PTS):</label>
            <input type="number" id="pts" required>

            <label for="ast">Assists (AST):</label>
            <input type="number" id="ast" required>

            <label for="reb">Rebounds (REB):</label>
            <input type="number" id="reb" required>

            <label for="season">Season:</label>
            <input type="number" id="season" required>

            <button type="submit">Submit</button>
        </form>

        <h2>Predictions:</h2>
        <pre id="predictions"></pre>
    </div>

    <script src="script.js"></script>
</body>
</html>


In [ ]:
body {
    font-family: Arial, sans-serif;
    background-color: #f4f4f4;
    margin: 0;
    padding: 20px;
}

.container {
    max-width: 600px;
    margin: auto;
    background: #fff;
    padding: 20px;
    border-radius: 8px;
    box-shadow: 0 0 10px rgba(0,0,0,0.1);
}

h1, h2 {
    color: #333;
}

label {
    margin-top: 10px;
    display: block;
}

input {
    width: 100%;
    padding: 8px;
    margin: 5px 0 15px;
    border: 1px solid #ccc;
    border-radius: 4px;
}

button {
    padding: 10px;
    background-color: #28a745;
    color: white;
    border: none;
    border-radius: 5px;
    cursor: pointer;
}

button:hover {
    background-color: #218838;
}

pre {
    background: #e9ecef;
    padding: 10px;
    border-radius: 5px;
}


In [ ]:
document.getElementById('statsForm').addEventListener('submit', async function(event) {
    event.preventDefault();

    const playerName = document.getElementById('playerName').value;
    const pts = parseFloat(document.getElementById('pts').value);
    const ast = parseFloat(document.getElementById('ast').value);
    const reb = parseFloat(document.getElementById('reb').value);
    const season = parseInt(document.getElementById('season').value);

    const formattedData = {
        SEASON: season,
        GP: 82,  // Example value
        GS: 0,   // Example value
        MIN: 30, // Example value
        FG_Made: pts * 0.5, // Example calculation
        FG_Att: pts, // Example calculation
        FG_Percent: (pts / (pts + ast + reb) * 100).toFixed(2), // Example FG%
        // You can add more calculated stats as needed
    };

    const response = await fetch('/predict', {
        method: 'POST',
        headers: {
            'Content-Type': 'application/json',
        },
        body: JSON.stringify(formattedData),
    });

    const predictions = await response.json();
    document.getElementById('predictions').textContent = JSON.stringify(predictions, null, 2);
});


In [ ]:
from flask import Flask, request, jsonify
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib

app = Flask(__name__)

# Prepare your data
data = {
    'SEASON': ['2015-16', '2016-17', '2017-18', '2018-19', '2019-20', '2020-21', '2021-22', '2022-23', '2023-24'],
    'GP': [80, 73, 75, 80, 73, 72, 74, 69, 79],
    'GS': [55, 59, 73, 80, 73, 72, 74, 69, 79],
    'MIN': [21.7, 27.9, 32.5, 31.3, 32.0, 34.6, 33.5, 33.7, 34.6],
    'FG': [(3.8, 7.5), (6.8, 11.7), (6.7, 13.5), (7.7, 15.1), (7.7, 14.7), (10.2, 18.0), (10.3, 17.7), (9.4, 14.8), (10.4, 17.9)],
    'FG%': [51.2, 57.8, 49.9, 51.1, 52.8, 56.6, 58.3, 63.2, 58.3],
    '3PT': [(0.4, 1.1), (0.6, 1.9), (1.5, 3.7), (1.0, 3.4), (1.1, 3.5), (1.3, 3.3), (1.3, 3.9), (0.8, 2.2), (1.1, 2.9)],
    '3P%': [33.3, 32.4, 39.6, 30.7, 31.4, 38.8, 33.7, 38.3, 35.9],
    'FT': [(1.9, 2.4), (2.6, 3.1), (3.5, 4.2), (3.6, 4.4), (3.4, 4.1), (4.8, 5.5), (5.1, 6.3), (4.9, 6.0), (4.5, 5.5)],
    'FT%': [81.1, 82.5, 85.0, 82.1, 81.7, 86.8, 81.0, 82.2, 81.7],
    'OR': [2.3, 2.9, 2.6, 2.9, 2.3, 2.8, 2.8, 2.4, 2.8],
    'DR': [4.7, 6.9, 8.1, 8.0, 7.5, 8.0, 11.0, 9.4, 9.5],
    'REB': [7.0, 9.8, 10.7, 10.8, 9.7, 10.8, 13.8, 11.8, 12.4],
    'AST': [2.4, 4.9, 6.1, 7.3, 7.0, 8.3, 7.9, 9.8, 9.0],
    'BLK': [0.6, 0.8, 0.8, 0.7, 0.6, 0.7, 0.9, 0.7, 0.9],
    'STL': [1.0, 0.8, 1.2, 1.4, 1.2, 1.3, 1.5, 1.3, 1.4],
    'PF': [2.6, 2.9, 2.8, 2.9, 3.0, 2.7, 2.6, 2.5, 2.5],
    'TO': [1.3, 2.3, 2.8, 3.1, 3.1, 3.1, 3.8, 3.6, 3.0],
    'PTS': [10.0, 16.7, 18.5, 20.1, 19.9, 26.4, 27.1, 24.5, 26.0]
}

df = pd.DataFrame(data)
df[['FG_Made', 'FG_Att']] = pd.DataFrame(df['FG'].tolist(), index=df.index)
df[['3PT_Made', '3PT_Att']] = pd.DataFrame(df['3PT'].tolist(), index=df.index)
df[['FT_Made', 'FT_Att']] = pd.DataFrame(df['FT'].tolist(), index=df.index)
df.drop(columns=['FG', '3PT', 'FT'], inplace=True)
df['SEASON'] = df['SEASON'].str.split('-', expand=True)[0].astype(int)

lower_bound = int(len(df) * 0.2)
upper_bound = int(len(df) * 0.9)
filtered_df = df.iloc[lower_bound:upper_bound]

features = ['SEASON', 'GP', 'GS', 'MIN', 'FG_Made', 'FG_Att', 'FG%', '3PT_Made', '3PT_Att', '3P%', 'FT_Made', 'FT_Att', 'FT%', 'OR', 'DR', 'BLK', 'STL', 'PF', 'TO']
targets = ['PTS', 'REB', 'AST']

models = {}
mse_scores = {}

for target in targets:
    X = filtered_df[features]
    y = filtered_df[target]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    mse_scores[target] = mse
    models[target] = model

    print(f"Mean Squared Error for {target}: {mse}")

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json

    features = np.array([[
        data['SEASON'],
        data['GP'],
        data['GS'],
        data['MIN'],
        data['FG_Made'],
        data['FG_Att'],
        data['FG_Percent'],
        0,  # Placeholder for 3PT_Made
        0,  # Placeholder for 3PT_Att
        0,  # Placeholder for 3P%
        0,  # Placeholder for FT_Made
        0,  # Placeholder for FT_Att
        0,  # Placeholder for FT%
        0,  # Placeholder for OR
        0,  # Placeholder for DR
        0,  # Placeholder for BLK
        0,  # Placeholder for STL
        0,  # Placeholder for PF
        0   # Placeholder for TO
    ]])

    predictions = {target: models[target].predict(features)[0] for target in targets}

    return jsonify(predictions)

if __name__ == '__main__':
    app.run(debug=True)
